In [1]:
## load required Python libraries
import os
import re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter
from nltk.util import ngrams
import random
from nltk import sent_tokenize
from random import randint
import math
import operator
from operator import itemgetter
import numpy as np
from nltk.metrics.distance import edit_distance
from random import sample
import difflib

In [2]:
#Φόρτωση των αρχείων του ευρωπαικού κοινοβουλίου
# Θέτουμε ως train set το 0.05% των συνολικών κειμένων με σκοπό τον προσδιορισμό 
# των unigrams και bigrams
files = os.listdir("C:/Users/User/Desktop/Text Engineering/en")
train_set = files[0:round(len(files)*0.05)]
test_set = files[round(len(files)*0.05):round(len(files)*0.05)+1]

In [3]:
#Φόρτωση των κειμένων των αρχείων σε μια list of lists, η πρώτη λίστα δηλαδή περιέχει 
#το κείμενο του πρώτου αρχείου
txt = []
for fname in train_set:
    with open('C:/Users/User/Desktop/Text Engineering/en/'+fname, 'r', encoding='utf8') as outfile:
        txt.append(outfile.read().replace('\n', ''))

In [4]:
#Καθαρισμός δεδομένων
#Εισάγουμε το q0->Start και το qf->end σε κάθε κείμενο
ntxt = []
for data in txt:
    data = re.sub('<[^>]*>', '', data)
    data = data.lower()
    data = 'q0' + ' ' +data+ ' '+ 'qf'
    ntxt.append(data)
txt = []
data = ''.join(ntxt)
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(data)
corpus_size = len(tokens)
len(tokens)

23355017

In [5]:
## Κάνοντας χρήση της συνάρτησης Counter της βιβλιοθηκης collections μετράμε τις συχνότητες εμφάνισης για κάθε λέξη
# στο Corpus
ntxt = []
count_tokens = dict(Counter(tokens))
len(count_tokens)

62467

In [6]:
#Αφαιρούμε από το λεξικό των μονογραμμάτων τις λέξεις που έχουν συχνότητα εμφάνισης μικρότερη από 10
# στην συνέχεια βρίσκουμε τις rare λέξεις (συχνότητα <=10)
unigrams = dict((k, v) for k, v in count_tokens.items() if v>10)
vocab_size = len(unigrams)
rare = dict((k, v) for k, v in count_tokens.items() if v<=10)

In [7]:
#Στα tokens, δηλαδή τις λέξεις, όποια λέξη άνήκει στις rare (συχνότητα <=10), την αντικαθιστούμε με rare 
for i in range(len(tokens)):
    if tokens[i] in rare.keys():
        tokens[i] = '*rare*'
count_tokens = dict()

In [8]:
#Δημιουργία δι-γραμμάτων με την χρήση της NLTK
bigrams = [ gram for gram in ngrams(tokens, 2)]
len(bigrams)

23355016

In [9]:
##εφαρμόζουμε την συνάρτηση Counter στα διγράμματα για να βρούμε τα μοναδικά διγράμματα
#καθώς και τις συχνότητες εμφάνισης τους
count_bigrams = dict(Counter(bigrams))
len(count_bigrams)

1971303

In [10]:
# βρίσκουμε τα διγράμματα που περιέχουν την λέξη rare
rare_bigrams = [k for k in count_bigrams.keys() if '*rare*' in k]
len(rare_bigrams)

16174

In [11]:
#Αφαιρούμε από τα μοναδικά διγράμματα τα διγράμματα που περιέχουν την λέξη rare
for item in rare_bigrams:
    count_bigrams.pop(item, None)
len(count_bigrams)
bigrams = []

In [12]:
# H παρακάτω συνάρτηση υπολογίζει την πιθανότητα διγραμμάτων δηλαδή p(wi|wi-1)
# κάνοντας χρήση του Laplace Smoothing
def get_bigram_prob(key):
    if key[0] in unigrams.keys():
        if key not in count_bigrams.keys():
            prob = (0+1)/(vocab_size+unigrams[key[0]])
        else:
            prob = (count_bigrams[key]+1)/(vocab_size+unigrams[key[0]])
    else:
        if len(key) == 2:
            if key[1] in unigrams.keys():
                prob = 0.4*(unigrams[key[1]]/sum(unigrams.values()))
            else:
                prob = 0.4/sum(unigrams.values())
        else:
            prob = 0.4/sum(unigrams.values())
    return prob

In [13]:
#Διάβασμα του τεστ αρχείου ανά γραμμές
sentences = []
for fname in test_set:
    with open('C:/Users/User/Desktop/Text Engineering/en/'+fname, 'r', encoding='utf8') as f:
        sentences += re.findall(r".*?[\.\!\?]+", f.read())

In [24]:
# τυχαία επιλογή πρότασης από το τεστ κείμενο
selected = random.choice(sentences)

In [25]:
selected

'Mr President, the remarks by my colleague, Mr Manders, did not come over very clearly via the interpretation.'

In [26]:
#καθαρισμός της πρότασης και σπάσιμο στα tokens της
tokenizer = RegexpTokenizer(r'\w+')
sel_tokens = tokenizer.tokenize(selected.lower())

In [27]:
sel_tokens

['mr',
 'president',
 'the',
 'remarks',
 'by',
 'my',
 'colleague',
 'mr',
 'manders',
 'did',
 'not',
 'come',
 'over',
 'very',
 'clearly',
 'via',
 'the',
 'interpretation']

In [78]:
#Τυχαία επιλογή των tokens στα οποία θα εισαχθεί θόρυβος. (μπορεί να είναι 1 έως όλες οι λέξεις της πρότασης) 
#Σε κάθε μία από τις επιλεγμένες λέξεις αντικαθιστούμε πάλι τυχαία ορισμένα ή και όλα τα 
#γράμματα της λέξης με κάποιο από τα υπόλοιπα γράμματα της αλφαβήτου ή και το κενό. 
def mis_spelling(sel_tokens):
    miss_words = int(input('Please set number of mis-spelled words: '))
    noise = []
    chosen_words = random.sample(range(0, len(sel_tokens)-1), 3)
    for i in range(0,len(sel_tokens)):
        if i in chosen_words:
            word = sel_tokens[i]
            letters = 'abcdefghijklmnopqrstuvwxyz'
            num_chars = randint(0,len(word)-1)
            inds = random.sample(range(0,len(word)-1),num_chars)
            let = random.sample(letters,num_chars)
            for i in range(0,num_chars):
                word = word[0:inds[i]]+let[i]+word[inds[i]+1:]
            noise.append(word)
        else:
            noise.append(sel_tokens[i])
    return noise

In [79]:
#Εκτύπωση των tokens της πρότασης εφόσον έχει εισαχθεί θόρυβος
noise = mis_spelling(sel_tokens)
noise

Please set number of mis-spelled words: 3


['mr',
 'president',
 'the',
 'remnrks',
 'by',
 'my',
 'colleague',
 'mr',
 'manders',
 'did',
 'not',
 'come',
 'over',
 'very',
 'clearly',
 'vma',
 'the',
 'interpretation']

In [80]:
# κάνοντας χρήση της συνάρτησης edit distance της βιβλιοθήκης NLTK
# και υπολογιζουμε για καθε λεξη τις 2 πιο κοντινές της
# θεωρούμε ως wi τις λέξεις στις οποίες έχει εισαχθεί θόρυβος
# θεωρούμε ως ti τις λέξεις που έρχονται ως αποτέλεσμα από την edit distance
w_in = noise
correct = dict()
for word in w_in:
    pos_dist = dict()
    for key in unigrams.keys():
        pos_dist[key] = edit_distance(word,key)
    correct[word] = sorted(pos_dist.items(), key=operator.itemgetter(1)) [0:2]

In [81]:
# Πιο συγκεκριμένα για κάθε λέξη επιστρέφονται 2 λέξεις όπως και παρατηρούμε στο παρακάτω λεξικό
correct

{'by': [('by', 0), ('bay', 1)],
 'clearly': [('clearly', 0), ('cheaply', 2)],
 'colleague': [('colleague', 0), ('colleagues', 1)],
 'come': [('come', 0), ('home', 1)],
 'did': [('did', 0), ('dip', 1)],
 'interpretation': [('interpretation', 0), ('interpretations', 1)],
 'manders': [('manders', 0), ('anders', 1)],
 'mr': [('mr', 0), ('or', 1)],
 'my': [('my', 0), ('m3', 1)],
 'not': [('not', 0), ('nut', 1)],
 'over': [('over', 0), ('mover', 1)],
 'president': [('president', 0), ('resident', 1)],
 'remnrks': [('remarks', 1), ('remark', 2)],
 'the': [('the', 0), ('tse', 1)],
 'very': [('very', 0), ('vary', 1)],
 'vma': [('vms', 1), ('via', 1)]}

In [82]:
## viterbi decoding
## το παρακάτω απόσπασμα κώδικα υλοποιεί τον αλγόριθμο viterbi
viterbi = []
t_prev = []
for i in range(0,len(w_in)):
    temp = []
    temp1 = []
    if i == 0:
        #V1 (t1) = P(t1|Start)P(w1|t1) 
        correct_w = correct[w_in[0]]
        for item in correct_w:
            temp.append( (item[0],-item[1] + np.log2(get_bigram_prob((item[0],'q0')))))
            temp1.append('q0')
        viterbi.append(temp)
        t_prev.append(temp1)
    else:
        #P(wk|tk)max_(tk-1){P(tk|tk-1)Vk-1(tk-1)}
        correct_w = correct[w_in[i]]
        for item in correct_w:
            find_max = []
            for tiprev in viterbi[i-1]:
                num = -item[1] + np.log2(get_bigram_prob((item[0],tiprev[0]))) + tiprev[1]
                find_max.append((tiprev[0],num))
            tt = max(find_max,key=itemgetter(1))
            temp1.append(tt[0])
            temp.append((item[0],tt[1]))
        t_prev.append(temp1)
        viterbi.append(temp)   

In [83]:
# Το αποτέλεσμα του παραπάνω κώδικα αποθηκεύεται σε μια list of tuples
# παρατηρούμε ότι για κάθε κοντινή λέξη (ti) μιας λέξης, ΄στην οποία έχει εισαχθεί θόρυβος, wi υπολογίζεται μια πιθανότητα
viterbi

[[('mr', -16.865999016160831), ('or', -17.178645401254993)],
 [('president', -22.150333607045219), ('resident', -29.990153894444223)],
 [('the', -30.446455676388382), ('tse', -37.535184796429967)],
 [('remarks', -40.271668894148391), ('remark', -44.047098711401723)],
 [('by', -56.434483071372526), ('bay', -55.669410366497203)],
 [('my', -71.566901343265357), ('m3', -69.81622878590909)],
 [('colleague', -84.336233527547691), ('colleagues', -84.013123104685036)],
 [('mr', -100.87912212084586), ('or', -99.606806005218871)],
 [('manders', -113.99037530868316), ('anders', -114.98692881445277)],
 [('did', -128.84241707507553), ('dip', -129.3700921061768)],
 [('not', -144.71763046696759), ('nut', -144.22490601152057)],
 [('come', -155.74907896633462), ('home', -158.3299806617415)],
 [('over', -170.94767747781967), ('mover', -171.12913411487952)],
 [('very', -186.93265214996919), ('vary', -185.34602036546556)],
 [('clearly', -197.56162522756591), ('cheaply', -201.73053432686419)],
 [('vms', -2

In [84]:
#υπολογισμός της μέγιστης πιθανότητας για την τελευταία λέξη w
# και εκτελώντας προς τα πίσω τον αλγόριθμο εντοπίζουμε το πιο πιθανό μονοπάτι
# το οποίο και εκτυπώνεται
max_el = max(viterbi[len(t_prev)-1],key=itemgetter(1))
max_index = viterbi[len(t_prev)-1].index(max_el)
path = []
path.append(max_el[0])
for j in range(0,len(t_prev)):
        path.append(t_prev[len(t_prev)-1-j][0])
most_probable_path = path[::-1][1:]
most_probable_path

['mr',
 'president',
 'the',
 'remarks',
 'bay',
 'm3',
 'colleagues',
 'or',
 'manders',
 'did',
 'not',
 'come',
 'over',
 'very',
 'clearly',
 'via',
 'the',
 'interpretation']

In [92]:
## calculate average distance (using edit_distance NLTK package)
## έχοντας πλέον την πιο πιθανή αλληλουχία λέξεων που προέκυψαν από την εκτέλεση του αλγορίθμου
# καθώς και την αρχική πρόταση πριν εισαχθεί σε αυτή ο θόρυβος
# υπολογίζουμε τις λέξεις που διόρθωσε σωστά ο viterbi context-sensitive corrector
# υπολογίζουμε τις λέξεις που προέβλεωε λάθος
# καθώς και την μέση oμοιότητα μεταξύ των λέξεων της μη-αλλοιωμένης πρότασης με τις λέξεις της πιο πιθανής αλλοιλουχίας
# που εντοπίσαμε
total = 0
num_of_correct_predicted_words = 0
for i in range(0,len(sel_tokens)):
    a = sel_tokens[i]
    b = most_probable_path[i]
    seq = difflib.SequenceMatcher(a=a.lower(), b=b.lower())
    #total += edit_distance(sel_tokens[i],most_probable_path[i])
    total += (1-seq.ratio())
    if seq.ratio() == 1:
        num_of_correct_predicted_words += 1
wrong_words = len(sel_tokens) - num_of_correct_predicted_words
avg_distance = total/len(sel_tokens)
print('Average difference between correct sentence and most propable sentence', avg_distance)
print('Correct predicted words: ', num_of_correct_predicted_words)
print('Wrong predicted words: ', wrong_words)

Average difference between correct sentence and most propable sentence 0.0695906432748538
Correct predicted words:  14
Wrong predicted words:  4


In [90]:
## baseline
## εφαρμόζουμε στην πρόταση με θόρυβο την συνάρτηση edit distance
## και στην συνέχεια αποθηκεύουμε σε λίστα
base = []
for item in noise:
    base.append(correct[item][1][0])
base

['or',
 'resident',
 'tse',
 'remark',
 'bay',
 'm3',
 'colleagues',
 'or',
 'anders',
 'dip',
 'nut',
 'home',
 'mover',
 'vary',
 'cheaply',
 'via',
 'tse',
 'interpretations']

In [93]:
## συγκρίνουμε το αποτέλεσμα της edit distance πάνω στην αλλοιωμένη πρόταση με την αρχική πρόταση.
total1 = 0
num_of_correct = 0
wrong = 0
for i in range(0,len(sel_tokens)):
    a = sel_tokens[i]
    b = base[i]
    seq = difflib.SequenceMatcher(a=a.lower(), b=b.lower())
    total1 += (1-seq.ratio())
    if seq.ratio() == 1:
        num_of_correct +=1
wrong = len(sel_tokens) - num_of_correct
avg = total1/len(sel_tokens)
print('Average difference between correct sentence and most propable sentence', avg)
print('Correct predicted words: ', num_of_correct)
print('Wrong predicted words: ', wrong)          

Average difference between correct sentence and most propable sentence 0.23499681949915038
Correct predicted words:  1
Wrong predicted words:  17
